<a href="https://colab.research.google.com/github/geopayme/AstroPhysics/blob/main/DLSFH_Entropy_AutoAnalysis_Patched.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ☀️ DLSFH Enhanced Diagnostic Notebook
Includes entropy mapping, SGCV coherence analysis, and NOAA sunspot timeline with fallback logic.

In [ ]:
# ✅ Install dependencies (Colab only)
!apt-get install -y tesseract-ocr
!pip install -q pytesseract


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from PIL import Image
import pytesseract
from datetime import datetime
from scipy.ndimage import sobel, generic_filter
from google.colab import files
import io, requests


In [ ]:
# 📤 Upload solar magnetogram
uploaded = files.upload()
image_path = next(iter(uploaded))
img = Image.open(image_path).convert('L')
img_array = np.array(img)


In [ ]:
# 🧠 Extract date from filename → OCR → manual
try:
    base = image_path.split('/')[-1]
    date_str = base[5:11]  # e.g., '250328'
    year = int('20' + date_str[:2])
    month = int(date_str[2:4])
    day = int(date_str[4:6])
    image_date = datetime(year, month, day)
    print(f"✅ Date from filename: {image_date.date()}")
except:
    try:
        text = pytesseract.image_to_string(img)
        date_parts = [int(p) for p in text.split()[0].split('-')]
        image_date = datetime(*date_parts)
        print(f"✅ Date from OCR: {image_date.date()}")
    except:
        manual = input("❗ Enter image date (YYYY-MM-DD): ")
        image_date = datetime.strptime(manual.strip(), "%Y-%m-%d")
        print(f"✅ Using manual date: {image_date.date()}")


In [ ]:
# 📈 NOAA Observed Solar Cycle (API → fallback CSV)
obs_url = "https://services.swpc.noaa.gov/json/solar-cycle/observed-solar-cycle-indices.json"
try:
    r = requests.get(obs_url)
    r.raise_for_status()
    data = r.json()
    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['time-tag'], errors='coerce')
    df = df[['date', 'ssn']].dropna()
    print(f"✅ NOAA API loaded ({len(df)} records)")
except:
    print("⚠️ NOAA API failed. Upload fallback CSV (columns: date, ssn)")
    uploaded_csv = files.upload()
    csv_name = next(iter(uploaded_csv))
    df = pd.read_csv(csv_name)
    df['date'] = pd.to_datetime(df['date'])


In [ ]:
# 📊 NOAA Sunspot Timeline
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(df['date'], df['ssn'], label='Smoothed Sunspot Number')
ax.axvline(image_date, color='red', linestyle='--', label=f'Image Date: {image_date.date()}')
ax.set_title('NOAA Solar Cycle Alignment')
ax.set_ylabel('Sunspot Number')
ax.legend()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# 🔬 Enhanced Entropy Extraction
h, w = img_array.shape
cx, cy = w // 2, h // 2
radius = min(cx, cy) - 10
y, x = np.ogrid[:h, :w]
mask = (x - cx)**2 + (y - cy)**2 <= radius**2

gx = sobel(img_array, axis=1)
gy = sobel(img_array, axis=0)
grad = np.hypot(gx, gy)
var = generic_filter(img_array, np.var, size=5)

grad_n = (grad - grad.min()) / (np.ptp(grad) + 1e-8)
var_n = (var - var.min()) / (np.ptp(var) + 1e-8)
entropy_map = np.zeros_like(img_array, dtype=float)
entropy_map[mask] = ((grad_n + var_n) / 2.0)[mask]

# Visualize
plt.figure(figsize=(6,6))
plt.imshow(entropy_map, cmap='inferno')
plt.title(f'Entropy Map ({image_date.date()})')
plt.axis('off')
plt.colorbar()
plt.show()


In [ ]:
# 🧬 DLSFH Projection + Report
angles = np.linspace(0, 2*np.pi, 21)[:-1]
radii = [radius * 0.4, radius * 0.8]
vec = []

for r in radii:
    for theta in angles[:10]:
        x_pt = int(cx + r * np.cos(theta))
        y_pt = int(cy + r * np.sin(theta))
        vec.append(entropy_map[y_pt, x_pt])

vec = np.array(vec)
σ_c = 0.07
collapsed = np.sum(vec >= σ_c)

print(f"=== DLSFH Entropy Report: {image_date.date()} ===")
print(f"Mean Entropy: {vec.mean():.4f} | Std Dev: {vec.std():.4f}")
print(f"Collapsed Nodes (≥ {σ_c}): {collapsed} / 20")
print(f"Inner Mean: {vec[:10].mean():.4f} | Outer Mean: {vec[10:].mean():.4f}")
print(f"Ring Divergence (Outer - Inner): {(vec[10:].mean() - vec[:10].mean()):.4f}")
if collapsed > 10:
    print('⚠️ Possible surface flux instability or decoherence.')
